In [1]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [109]:
categories = ['airlines', 'celebrities', 'colleges', 'fastfood', 
              'leagues', 'news', 'streamingplatforms', 'techgiants']

subCategories = [['output_AmericanAir', 'output_Delta', 'output_easyJet', 'output_JetBlue'], 
                 ['output_amyschumer', 'output_ArianaGrande', 'output_chrissyteigen', 'output_jimmyfallon'], 
                 ['output_DukeU', 'output_Harvard', 'output_MIT', 'output_Princeton'], 
                 ['output_Arbys', 'output_BurgerKing', 'output_ChickFilA', 'output_ChipotleTweets'], 
                 ['output_IPL', 'output_MLB', 'output_MLS', 'output_NBA'], 
                 ['output_BBCWorld', 'output_CNN', 'output_FoxNews', 'output_HuffPost'], 
                 ['output_ABCNetwork', 'output_AppleMusic', 'output_disneyplus', 'output_HBO'], 
                 ['output_amazon', 'output_facebook', 'output_fitbit', 'output_IBM']]

In [114]:
sizeOfSets = 400

categCount = 0
for i in range(len(categories)):
    for j in subCategories[i]:
        categCount += 1
        
categCount *= sizeOfSets

val = [None] * categCount # Keep value (sizeOfSets * no. of categories)
sizeOfTweetContent = 75

print(len(val)) # No. Of Posts

12800


In [115]:
def inputDataInDataframe():
    overCount= 0
    for i in range(len(categories)):
        for j in subCategories[i]:
            direc:str = '/Users/sameernawaz/AnacondaProjects/Datasets/tweets/'+categories[i]+'/'+j+'.csv'
            with open(direc, 'r') as csv_file:
                csv_reader = csv.reader(csv_file)
                count = 0
                for line in csv_reader:
                    if len(line[6]) > sizeOfTweetContent:
                        val[overCount] = line
                        count += 1
                        overCount += 1
                        if count >= sizeOfSets :
                            break

In [116]:
inputDataInDataframe()

In [117]:
df = pd.DataFrame(val, columns = ['date', 'username', 'to', 'replies', 
                                  'retweets', 'favorites', 'text', 'geo', 
                                  'mentions', 'hashtags', 'id', 'permalink']) 

In [118]:
# Once data added in dataframe, Shuffle it
df = df.sample(frac=1).reset_index(drop=True)
# Add a unique id (rec_id) for all tweets
df = df.assign(rec_id=np.arange(len(df))).reset_index(drop=True)

In [119]:
df = df[['rec_id', 'username', 'text']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12800 entries, 0 to 12799
Data columns (total 3 columns):
rec_id      12800 non-null int64
username    12800 non-null object
text        12800 non-null object
dtypes: int64(1), object(2)
memory usage: 400.0+ KB


In [120]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['text']))
len(norm_corpus)

12800

In [121]:
# Extract TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(12800, 25271)

In [122]:
# Compute Pairwise Document Similarity

from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,12790,12791,12792,12793,12794,12795,12796,12797,12798,12799
0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.028164
1,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,1.0,0.0,0.0,0.045639,0.016082,0.0,0.028028,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.024559,0.016572,0.017659,0.000000
3,0.0,0.0,0.0,1.0,0.0,0.000000,0.035562,0.0,0.019943,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.000000,0.089032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [123]:
# Get list of post IDs

posts_list = df['rec_id'].values
posts_list, posts_list.shape

(array([    0,     1,     2, ..., 12797, 12798, 12799]), (12800,))

In [124]:
posts_idx = np.where(posts_list == 3997)[0][0]
posts_idx

3997

In [125]:
posts_similarities = doc_sim_df.iloc[posts_idx].values
posts_similarities

array([0.        , 0.        , 0.10489771, ..., 0.01281918, 0.01366   ,
       0.        ])

In [126]:
# Get top 5 similar post IDs

similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
similar_posts_idxs

array([ 8745, 12431, 10785,  6713,  8249])

In [127]:
similar_posts = posts_list[similar_posts_idxs]
similar_posts

array([ 8745, 12431, 10785,  6713,  8249])

In [128]:
def relevant_posts(postId, posts = posts_list, doc_sims = doc_sim_df):
    # find post id
    posts_idx = np.where(posts == postId)[0][0]
    # get posts similarities
    posts_similarities = doc_sims.iloc[posts_idx].values
    # get top 5 similar post IDs
    similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
    # get top 5 posts
    similar_posts = posts[similar_posts_idxs]
    # return the top 5 posts
    return similar_posts

In [141]:
post_id = 10101

print('-----Input Post-----')
print((df.loc[df['rec_id'] == post_id]['text']).values[0])
print('\n-----Relevant Posts-----')
for i in relevant_posts(postId = post_id, posts = posts_list, doc_sims = doc_sim_df):
    print("=>",(df.loc[df['rec_id'] == i]['text']).values[0])

-----Input Post-----
Former Navy Secretary Richard Spencer penned an op-ed in The Washington Post about his clashes with President Trump over the case of convicted war criminal Eddie Gallagher, which led to Spencer's resignation.

-----Relevant Posts-----
=> In his first interview since being ousted as Navy secretary, Richard Spencer told CBS that President Donald Trump's decision to block the Pentagon's review of convicted war criminal Eddie Gallagher sends a message "that you can get away with things."
=> Secretary of Defense Mark Esper has asked Navy Secretary Richard Spencer to resign over the demotion of a Navy SEAL convicted of war crimes.
=> Recently ousted Navy Secretary Richard Spencer detailed President Trump's intervention in a war crimes case in a Washington Post op-ed, calling the President's actions "shocking and unprecedented"
=> Recently ousted Navy Secretary Richard Spencer detailed President Trump's intervention in a war crimes case in a Washington Post op-ed Wednesda